In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00


In [ ]:
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.9 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=3f8ae9cb7096ec68f25458cc92ac60c7b2b65ed7c1bd48f2226f86f419cd64f7
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim import Adam
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
import string
from googletrans import Translator
import random

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Elsevier /Pranita Project /df_clean (1).csv')
data.head()

,Unnamed: 0,Table_title,concept_1,predicate,concept_2
0,0,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,mild elevation ALT/AST level
1,1,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,moderate elevation GGT level
2,2,Hepatitic and cholestatic/obstructive liver fu...,biliary obstruction,has clinical finding,marked elevation ALP level
3,3,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,marked elevation ALT/AST level
4,4,Hepatitic and cholestatic/obstructive liver fu...,hepatitis,has clinical finding,mild elevation GGT level


In [ ]:
def clean_string(text):
    words = text.split()
    clean_words = []
    for word in words:
        word = word.translate(str.maketrans('', '', string.punctuation + string.digits))
        if len(word) > 1:
            clean_words.append(word)
    clean_text = ' '.join(clean_words)
    return clean_text

data = data.fillna('')

translator = Translator()

In [ ]:
# Function to translate text from English to Spanish and back to English
def translate_text(text):
    # Initialize the translator
    translator = Translator(service_urls=['translate.google.com'])

    # Translate text from English to Spanish
    translation = translator.translate(text, dest='es')
    spanish_text = translation.text

    # Translate the Spanish text back to English
    back_translation = translator.translate(spanish_text, dest='en')
    english_text = back_translation.text

    #return english_text
    return english_text if english_text is not None else text


In [ ]:
# Create a label encoder
encoder = LabelEncoder()

# Fit the encoder on the predicate column
encoder.fit(data['predicate'])

# Transform the predicate column
predicate = encoder.transform(data['predicate'])

In [ ]:
predicate

array([11, 11, 11, ..., 14, 14, 14])

In [ ]:

# Set the number of classes
num_classes = 18

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=num_classes)

# Create a new DataFrame to store the translated samples
translated_data = pd.DataFrame(columns=data.columns)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [ ]:
translated_data

,Unnamed: 0,Table_title,concept_1,predicate,concept_2


In [ ]:
# Iterate over each unique class in the predicate column
for label in data['predicate'].unique():
    # Get the rows corresponding to the current class
    class_rows = data[data['predicate'] == label]

    # Determine the number of additional samples needed for the minority class
    class_count = len(class_rows)
    max_count = data['predicate'].value_counts().max()
    additional_samples = max_count - class_count

    # Translate the text for additional samples
    if additional_samples > 0:
        # Translate each row to Spanish and back to English
        for i in range(additional_samples):
            random_row = random.choice(class_rows.index)
            row = class_rows.loc[random_row].copy()

            # Translate the text columns to Spanish
            row['Table_title'] = translate_text(row['Table_title'])
            row['concept_1'] = translate_text(row['concept_1'])
            row['concept_2'] = translate_text(row['concept_2'])

            # Append the translated row to the translated_data DataFrame
            translated_data = translated_data.append(row)

TypeError: ignored

In [ ]:
element = data.iloc[0, 2]
print(element)
type(element)

biliary obstruction


str

In [ ]:
data['Table_title'].dtype

dtype('O')

In [ ]:
#debugging
c = 0
# Iterate over each unique class in the predicate column
for label in data['predicate'].unique():
    # Get the rows corresponding to the current class
    class_rows = data[data['predicate'] == label]

    # Determine the number of additional samples needed for the minority class
    class_count = len(class_rows)
    max_count = data['predicate'].value_counts().max()
    additional_samples = max_count - class_count

    # Translate the text for additional samples
    if additional_samples > 0:
        # Translate each row to Spanish and back to English
        for i in range(additional_samples):
            random_row = random.choice(class_rows.index)
            row = class_rows.loc[random_row].copy()

           #print(row.dtype)

            print(row['Table_title'])


Streaming output truncated to the last 5000 lines.
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical and investigation findings in the immunobullous disorders
Clinical an

In [ ]:
# Concatenate the translated_data with the original data
data = pd.concat([data, translated_data], ignore_index=True)

# Convert the columns to strings
data['Table_title'] = data['Table_title'].astype(str).apply(clean_string)
data['concept_1'] = data['concept_1'].astype(str).apply(clean_string)
data['concept_2'] = data['concept

In [ ]:
# basic code for just translating

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
import torch
from torch.optim import Adam
from googletrans import Translator
import string

data = pd.read_csv('/content/drive/MyDrive/Elsevier /Pranita Project /df_clean (1).csv')
data.head()

def clean_string(text):
    words = text.split()
    clean_words = []
    for word in words:
        word = word.translate(str.maketrans('', '', string.punctuation + string.digits))
        if len(word) > 1:
            clean_words.append(word)
    clean_text = ' '.join(clean_words)
    return clean_text

data = data.fillna('')

translator = Translator()

# Perform machine translation to generate additional samples
translated_data = data.copy()
translated_data['Table_title'] = translated_data['Table_title'].apply(lambda x: translator.translate(x, dest='fr').text)
translated_data['concept_1'] = translated_data['concept_1'].apply(lambda x: translator.translate(x, dest='fr').text)
translated_data['concept_2'] = translated_data['concept_2'].apply(lambda x: translator.translate(x, dest='fr').text)
translated_data['predicate'] = data['predicate']  # Keep the original predicate column

# Concatenate the original and translated data
augmented_data = pd.concat([data, translated_data], ignore_index=True)

encoder = LabelEncoder()
encoder.fit(augmented_data['predicate'])
predicate = encoder.transform(augmented_data['predicate'])
num_classes = 18

categorical_features = ['Table_title', 'concept_1', 'concept_2']
ordinal_encoder = OrdinalEncoder()
data_encoded = ordinal_encoder.fit_transform(augmented_data[categorical_features])

tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=num_classes)

text = augmented_data['Table_title'].astype(str) + ' ' + augmented_data['concept_1'].astype(str) + ' ' + augmented_data['concept_2'].astype(str)
inputs = tokenizer(text.tolist(), return_tensors="pt", padding=True, truncation=True)

labels = torch.tensor(predicate)

input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

batch_size = 8
dataset = TensorDataset(input_ids, attention_mask, labels)
dataloader = DataLoader(dataset, batch_size=batch_size)

learning_rate = 1e-5
gradient_accumulation_steps = 4

optimizer = Adam(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

model.train()
for epoch in range(3):
    for i, (input_ids, attention_mask, labels) in enumerate(dataloader):
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

# Rest of your code...
